In [1]:
import numpy as np  # Librería para aplicar álgebra lineal
import pandas as pd # Para manejar los datos (datasets)
import matplotlib.pyplot as plt # Para las visualizaciones
import seaborn as sns # Visualizaciones más fancy


#Iremos agregando las bibliotecas a medida que las necesitemos

In [2]:
#Datos de entrada

#Se especifican los tipo de datos para mejorar la performance
eventos = pd.read_csv('../TP2018/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv', 
                      dtype={"timestamp": object,#intenté hacerla datetime y no anduvo 
                             "event": 'category',
                             "person":object,
                             "url":object,
                             "sku":object,
                             "model":object,
                             "condition":'category',
                             "storage":object,
                             "color":'category',
                             "skus":object,
                             "search_term":object,
                             "staticpage":object,
                             "campaign_source":object,
                             "search_engine":object,
                             "channel":object,
                             "new_vs_returning":'category',
                             "city":object,
                             "region":object,
                             "country":object,
                             "device_type":object,
                             "screen_resolution":object,
                             "operating_system_version":object,
                             "browser_version":object})

labels = pd.read_csv('../TP2018/fiuba-trocafone-tp2-final-set/labels_training_set.csv',
                        dtype={"person":object, "label":bool})

clientesAEvaluar = pd.read_csv('../TP2018/fiuba-trocafone-tp2-final-set/trocafone_kaggle_test.csv')

In [3]:
#Agregamos algunos features a los existentes
eventos['timestamp'] =  pd.to_datetime(eventos['timestamp'])
eventos[['marca','modelo']] = eventos['model'].dropna().str.split(' ',n=1,expand=True)
eventos['weekday'] = eventos['timestamp'].dt.day_name()
eventos['hour'] = eventos['timestamp'].dt.hour
eventos['month'] = eventos['timestamp'].dt.month
eventos['day'] = eventos['timestamp'].dt.day

In [4]:
#Creo períodos de tiempo donde ya asignamos a qué mes pertenecen
eventos['principio_de_mes'] = ((eventos['day'] < 10) * eventos['month'])
eventos['fin_de_mes'] = ((eventos['day'] > 20) * eventos['month'])
eventos['mitad_de_mes'] = (((eventos['day'] > 10)  & (eventos['day'] < 21)) * eventos['month'])
#eventos.loc[1:10,['principio_de_mes','mitad_de_mes','fin_de_mes']]

### Vamos creando features: 

In [5]:
#Features en base al total de cada evento
totalEvento = eventos.groupby('person')['event'].value_counts().unstack().reset_index()
totalEvento.rename(lambda x: x if x == 'person' else 'total ' + x, axis = 1, inplace=True)

#Features en base a la frecuencia de cada evento sobre el total (frecuencia relativa)
frecEvento = eventos.groupby('person')['event'].value_counts(normalize=True).unstack().reset_index()
frecEvento.rename(lambda x: x if x == 'person' else 'frec ' + x, axis = 1, inplace=True)

#features en base a la frecuencia relativa de la marca de los dispositivos consultados
frecMarca = eventos.groupby('person')['marca'].value_counts(normalize=True).unstack().reset_index()
frecMarca.rename(lambda x: x if x == 'person' else 'frec ' + x, axis = 1, inplace=True)

#features en base a la cantidad (frecuencia total) de eventos por mes
totalMes = eventos.groupby('person')['month'].value_counts().unstack().reset_index()
totalMes.rename({1:'total_enero',2:'total_febrero',3:'total_marzo',4:'total_abril',5:'total_mayo'}, 
                inplace=True, axis=1)

#features en base a la cantidad de eventos en períodos de aprox 10 días (principio, mitad y fin de mes)
totalPrincipioMes = eventos.groupby('person')['principio_de_mes'].value_counts().unstack().reset_index().head().drop(0,axis=1)
totalPrincipioMes.rename(lambda x: x if x == 'person' else 'total principio_de_mes ' + str(x), axis = 1, inplace=True)

totalMitadMes = eventos.groupby('person')['mitad_de_mes'].value_counts().unstack().reset_index().head().drop(0,axis=1)
totalMitadMes.rename(lambda x: x if x == 'person' else 'total mitad_de_mes ' + str(x), axis = 1, inplace=True)

totalFinMes = eventos.groupby('person')['fin_de_mes'].value_counts().unstack().reset_index().head().drop(0,axis=1)
totalFinMes.rename(lambda x: x if x == 'person' else 'total fin_de_mes ' + str(x), axis = 1, inplace=True)

#features en base a la frecuencia relativa de los dispositivos desde los que se accede
frecDispositivo = eventos.groupby('person')['device_type'].value_counts(normalize=True).unstack().reset_index()
frecDispositivo.rename(lambda x: x if x == 'person' else 'frec ' + x, axis=1, inplace=True)

#features en base a la frecuencia relativa de los colores de los teléfonos vistos
frecColor = eventos.groupby('person')['color'].value_counts(normalize=True).unstack().reset_index()
frecColor.rename(lambda x: x if x == 'person' else 'frec color ' + x, axis=1, inplace=True)

#features en base a la frecuencia relativa de los estados de los teléfonos consultados
frecCondicion = eventos.groupby('person')['condition'].value_counts(normalize=True).unstack().reset_index()
frecCondicion.rename(lambda x: x if x == 'person' else 'frec ' + x, axis=1, inplace=True)

#features en base al la frecuencia relativa del día de semana en que se consultó
frecDiaSemana = eventos.groupby('person')['weekday'].value_counts(normalize=True).unstack().reset_index().fillna(0)
frecDiaSemana.rename(lambda x: x if x == 'person' else 'frec ' + x, axis=1, inplace=True)

#features en base al día (a revisar, capaz convenga hacer algo en base al més y al día, o separar en quincenas)
frecNroDia = eventos.groupby('person')['day'].value_counts(normalize=True).unstack().reset_index()
frecNroDia.rename(lambda x: x if str(x) == 'person' else 'frec dia ' + str(x), axis=1, inplace=True)

#Creamos los features del almacenamiento interno de los dispositivos consultados
frecAlmacenamiento = eventos.groupby('person')['storage'].value_counts(normalize=True).unstack().reset_index()
frecAlmacenamiento.rename(lambda x: x if x == 'person' else 'frec ' + x, axis=1, inplace=True)


In [6]:
#Empezamos a hacer el merge
#No hay que olvidarse de hacer outer left joins, si no se perderán filas
#Empezamos con features con todas las filas (todos los clientes tienen eventos)
features = frecEvento.merge(totalEvento, how='left', on='person')
features = features.merge(frecMarca, how='left', on='person')
features = features.merge(totalMes, how='left', on='person')
features = features.merge(frecDispositivo, how='left', on='person')
#features = features.merge(frecColor, how='left', on='person')
features = features.merge(frecCondicion, how='left', on='person')
features = features.merge(frecDiaSemana, how='left', on='person')
features = features.merge(totalPrincipioMes, how='left', on='person')
features = features.merge(totalMitadMes, how='left', on='person')
features = features.merge(totalFinMes, how='left', on='person')
features = features.merge(frecNroDia, how='left', on='person') #Lo reemplazo por los períodos en los meses?
features = features.merge(frecAlmacenamiento, how='left', on='person')

features.head(2)

,person,frec ad campaign hit,frec brand listing,frec checkout,frec conversion,frec generic listing,frec lead,frec search engine hit,frec searched products,frec staticpage,...,frec dia 30,frec dia 31,frec 128GB,frec 16GB,frec 256GB,frec 32GB,frec 4GB,frec 512MB,frec 64GB,frec 8GB
0,0008ed71,NaN,NaN,0.500000,NaN,0.166667,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.666667,NaN,NaN,0.333333,NaN
1,00091926,0.033482,0.055804,0.004464,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.035714,0.128342,0.278075,0.026738,0.352941,NaN,NaN,0.213904,NaN


## Preparación de algoritmos clasificadores y otras yerbas

In [7]:
#Cargamos las bibliotecas para usar y validar Random Forest y XGBoost
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
import xgboost as xgb

#Para preparar sets de entrenamiento y prueba
from sklearn.model_selection import train_test_split  

#Nos traemos bibliotecas para medir la performance
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [8]:
'''Tomamos "prestada" una función para presentar prolijamente los n features más importantes.
La función fue tomada casi literal de:
    https://github.com/Featuretools/predict-next-purchase/blob/master/utils.py
'''
def feature_importances(model, features, n=10):
    importances = model.feature_importances_
    zipped = sorted(zip(features, importances), key=lambda x: -x[1])
    for i, f in enumerate(zipped[:n]):
        print("%d: Feature: %s, %.3f" % (i+1, f[0], f[1]))

    return [f[0] for f in zipped[:n]]

In [9]:
#Reemplazamos NaNs por ceros
features = features.fillna(0)

#Cargamos X e y para trabajar con Random Forest (y otros algoritmos luego).
X = pd.merge(features, labels, on='person')
X.drop('person', axis=1, inplace=True)
X = X.fillna(0)
y = X.pop('label')

#Ahora hacemos el split de los datos a testear
#Hacemos una división de set de test y entrenamiento, y le ponemos una semilla arbitraria para poder replicarlo
seed = 1234
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=seed)

### Creamos los clasificadores

In [10]:
#XGBoost básico, sólo con la semilla para poder replicarlo
xgbBasico = xgb.XGBClassifier(seed=seed)

In [11]:
#Random con la semilla para poder replicarlo
clfRF1 = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini', max_depth=None, 
                                max_features='auto', max_leaf_nodes=None, min_samples_leaf=1, 
                                min_samples_split=2,min_weight_fraction_leaf=0.0,n_estimators=500, 
                                n_jobs=-1, oob_score=False, random_state=seed, verbose=0, warm_start=False)

## Preparamos la clasificación a subir

In [12]:
## Preparamos los datos que necesitamos clasificar
datosTest = clientesAEvaluar.merge(features, on='person',how='inner')

In [13]:
#Hacemos predicciones con el modelo de Random Forest
clfRF1.fit(X,y)
prediccionRandomForest = clfRF1.predict_proba(X=datosTest.drop('person', axis=1).fillna(0))

In [14]:
#Predicciones con el modelo básico de XGBoost entrenado con los datos sin ordenar
xgbBasico.fit(X,y)
prediccionXGBoostBasico = xgbBasico.predict_proba(data=datosTest.drop('person', axis=1).fillna(0))

In [15]:
#ensamble, en lugar del promedio, asignarle pesos
clientesAEvaluar['label'] = (prediccionXGBoostBasico[:,[1]] * 0.6 + prediccionRandomForest[:,[1]] * 0.4)
clientesAEvaluar.to_csv('Entregas/entrega_XGBoostBasico y RF 2.csv',index=False)


### Con esta entrega generamos nuestro mejor resultado: 0.86189